## Clustering via K-Medoids

Courtesy of Georgia Tech's CSE 6040, this is one of the exam questions that I wrote. I had hard time finding materials on K-Medoids as it is being overshadowed by K-Means. However, I truly think this is a great method especially when working with dataset with outliers so I decided to expland on the codes I wrote during the exam and tackled another clustering problem.

In this project, it will focus on the concept of "k-medoids". Both the k-means and k-medoids algorithms are partitional (breaking the dataset up into groups). K-means attempts to minimize the total squared error from a central position in each cluster (centroids), while k-medoids minimizes the sum of dissimilarities between objects labeled to be in a cluster and one of the objects designated as the representative of that cluster (medoids). In contrast to the k-means algorithm, k-medoids chooses medoids from the objects in the set, instead of a central, average position in the feature space.

The possible choice of the dissimilarity function is very rich but here, we will use the more general, __Minkowski Distance Function__ of a given order.

The algorithm that we will be implementing is called the __PAM algorithm__ (Partitioning Around Medoids, (Kaufman and Rousseeuw 1990)). It can be summarized as follows:
1. __Initialize__: randomly select $k$ of the $m$ data points as the medoids
2. __Assignment step__: Associate each data point to the closest medoid.
3. __Update step__: For each medoid $j$ and each data point $i$ associated to $j$ swap $j$ and $i$ and compute the total cost of the configuration (that is, the average dissimilarity of $i$ to all the data points associated to $j$). Select the medoid $j$ with the lowest cost of the configuration.
Repeat alternating steps 2 and 3 until there is __no change__ in the assignments.

In the code that follows, it will be convenient to use our usual "data matrix" convention, that is, in the data matrix $X$, each row, $\hat{x}$, is one of  $m$  observations and each column, $x$, is one of $f$  features. However, we will not need a dummy column of ones since we are not fitting a function.


\begin{equation*}
X \equiv
\begin{bmatrix}
x_{0,0} \cdots x_{0,f-1} \\
\vdots\\
x_{m-1,0} \cdots x_{m,f-1}\\
\end{bmatrix}
\end{equation*}

We will use a copy of UCI ML Wine Data Set dataset is downloaded and modified to fit standard format from:
https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data

In [74]:
# Imports
import pandas as pd
import numpy as np
import os

In [75]:
# Dataset
from sklearn.datasets import load_wine
data = load_wine()

The features or predictors are:

In [76]:
features = data.feature_names

print(features)

['alcohol', 'malic_acid', 'ash', 'alcalinity_of_ash', 'magnesium', 'total_phenols', 'flavanoids', 'nonflavanoid_phenols', 'proanthocyanins', 'color_intensity', 'hue', 'od280/od315_of_diluted_wines', 'proline']


In [77]:
df = pd.DataFrame(data.data,columns = features)
df.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0


The classification are:

In [81]:
classes=np.unique(data.target).tolist()
classes

[0, 1, 2]

The actual classification for all datapoints is as below:

In [85]:
wine_type = data.target

wine_type

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2])

In [86]:
df.describe()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
count,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000
mean,13.000618,2.336348,2.366517,19.494944,99.741573,2.295112,2.029270,0.361854,1.590899,5.058090,0.957449,2.611685,746.893258
std,0.811827,1.117146,0.274344,3.339564,14.282484,0.625851,0.998859,0.124453,0.572359,2.318286,0.228572,0.709990,314.907474
min,11.030000,0.740000,1.360000,10.600000,70.000000,0.980000,0.340000,0.130000,0.410000,1.280000,0.480000,1.270000,278.000000
25%,12.362500,1.602500,2.210000,17.200000,88.000000,1.742500,1.205000,0.270000,1.250000,3.220000,0.782500,1.937500,500.500000
50%,13.050000,1.865000,2.360000,19.500000,98.000000,2.355000,2.135000,0.340000,1.555000,4.690000,0.965000,2.780000,673.500000
75%,13.677500,3.082500,2.557500,21.500000,107.000000,2.800000,2.875000,0.437500,1.950000,6.200000,1.120000,3.170000,985.000000
max,14.830000,5.800000,3.230000,30.000000,162.000000,3.880000,5.080000,0.660000,3.580000,13.000000,1.710000,4.000000,1680.000000


We noticed there is no missing data but not all are on the same scale. So next, let's scale the data in each column.

In [87]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

df.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,0.842105,0.191700,0.572193,0.257732,0.619565,0.627586,0.573840,0.283019,0.593060,0.372014,0.455285,0.970696,0.561341
1,0.571053,0.205534,0.417112,0.030928,0.326087,0.575862,0.510549,0.245283,0.274448,0.264505,0.463415,0.780220,0.550642
2,0.560526,0.320158,0.700535,0.412371,0.336957,0.627586,0.611814,0.320755,0.757098,0.375427,0.447154,0.695971,0.646933
3,0.878947,0.239130,0.609626,0.319588,0.467391,0.989655,0.664557,0.207547,0.558360,0.556314,0.308943,0.798535,0.857347
4,0.581579,0.365613,0.807487,0.536082,0.521739,0.627586,0.495781,0.490566,0.444795,0.259386,0.455285,0.608059,0.325963


There are 13 predictors which might not be ideal for visualization and fitting of the clustering model since many of them might be highly correlated. Therefore, we will employ the Principal Component Analysis (PCA) to transform 13-dimensional data into 3-dimensional data while keeping the significane of those predictors.

In [88]:
from sklearn.decomposition import PCA


pca = PCA(n_components=3)

principalComponents = pca.fit_transform(df)

PCAdf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2','principal component 3'])

PCAdf.head()

,principal component 1,principal component 2,principal component 3
0,-0.706336,-0.253193,0.024093
1,-0.484977,-0.008823,-0.280482
2,-0.521172,-0.189187,0.196217
3,-0.821644,-0.580906,0.081110
4,-0.202546,-0.059467,0.300240


Let's extract the data points as a two-dimensional NumPy ndarray, points, and the labels as a one-dimensional NumPy ndarray, labels. Note that the k-medoids algorithm we will implement should not reference labels but these are for accuracy check later.

In [89]:
points = principalDf[list(principalDf.columns)].values
labels = wine_type
m, f = points.shape
k = 3

Let's visualize the clusters on a 3D graph:

In [90]:
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

trace1 = go.Scatter3d(
    x=PCAdf['principal component 1'],
    y=PCAdf['principal component 2'],
    z=PCAdf['principal component 3'],
    mode='markers',
    marker=dict(
        size=6,
        color=wine_type,                # set color to an array/list of desired values
        colorscale='Viridis',   # choose a colorscale
        opacity=0.8
    )
)

data = [trace1]
layout = go.Layout(
    scene = dict(
        xaxis = dict(
            title='Principal Component 1'),
        yaxis = dict(
            title='Principal Component 2'),
        zaxis = dict(
            title='Principal Component 3'),),
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    ),
    showlegend=False
)
fig = go.Figure(data=data, layout=layout)
iplot(fig)

### Medoid Initialization:
To start the algorithm, you need an initial guess. Let's randomly choose  k  observations from the data.

Creating function, init_medoids(X, k), so that it randomly selects  k  of the given observations to serve as medoids. It should return a NumPy array of size $k \times d$, where d is the number of columns of X. You may use modules and methods from NumPy or the Python Standard Library.

In [213]:
def init_medoids(X, k):
    """
    Randomly samples k observations from X as medoids.
    Returns these medoids as a numpy array, with shape (k,f).
    """
    from numpy.random import choice
    from numpy.random import seed
    
    seed(1708)
    samples = choice(len(X), size=k, replace=False)
    return X[samples, :]

In [214]:
medoids_initial = init_medoids(points, k)

print("Initial medoids:\n {}".format(medoids_initial))

Initial medoids:
 [[ 0.04979031  0.55826515  0.18861497]
 [ 0.32077703  0.40796418 -0.06989672]
 [ 0.701764   -0.51350498  0.29390996]]


### Computing the distances
https://en.wikipedia.org/wiki/Minkowski_distance
    
Implementing a function that computes a distance matrix,  $S=(s_{ij})$  such that  $s_{ij}=d^{p}_{ij}$  is the $p^{th}$ power of the Minkowski distance (order $p$) from point  $x_i$  to medoid  $\mu_j$ . It should return a NumPy matrix S with shape (m,k).

The $p^{th}$ order Minkowski Distance between two points, $x$ and $\mu$ is given by:

$$\begin{equation*}
d_{ij} = \bigg(\sum^{f}_{a=1}\mid x_{ia} - \mu_{ja} \mid ^p\bigg)^{\frac{1}{p}} \textrm{,} \quad  0 < i < (m-1) \textrm{,} \quad  0 < j < (k-1)
\end{equation*}$$

Thus, for the elements of $S$,

$$\begin{equation*}
s_{ij} = d^{p}_{ij} = \sum^{f}_{a=1}\mid x_{ia} - \mu_{ja} \mid ^p
\end{equation*}$$

Minkowski distance is typically used with $p$ being 1 or 2, which correspond to the __Manhattan distance__ and the __Euclidean distance__, respectively, but $p$ is left as an input parameter in this function.


In [215]:
def compute_d_p(X, medoids, p):
    m = len(X)
    medoids_shape = medoids.shape
    # If a 1-D array is provided, 
    # it will be reshaped to a single row 2-D array
    if len(medoids_shape) == 1: 
        medoids = medoids.reshape((1,len(medoids)))
    k = len(medoids)
    
    S = np.empty((m, k))
    
    for i in range(m):
        d_i = np.linalg.norm(X[i, :] - medoids, ord=p, axis=1)
        S[i, :] = d_i**p

    return S

In [216]:
S = compute_d_p(points, medoids_initial, 2)

### Cluster Assignment
Implementing a function that acts on the $p^{th}$ order distance matrix to assign a "cluster label" to each point using the minimum distance to find the "most similar" medoid.

That is, consider the  $m \times k$,  $p^{th}$ power distance matrix  S .

For each point, indicated by row index $i$ , if  $s_{ij}$  is the minimum distance for point  $i$ , then the index  $j+1$  is  $i$ 's cluster label.

In other words, your function should return a one-dimensional array,  $y$,  of length  $m$  such that

\begin{equation*}
y_{i} = \underset{j∈\{0,…,k−1\}}{\operatorname{argmin}}s_{ij} + 1
\end{equation*}

In [217]:
def assign_cluster_labels(S):
    return np.argmin(S, axis=1)

First run of cluster label assignment:

In [218]:
labels = assign_cluster_labels(S)

labels

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0,
       0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 2, 0, 0, 1, 0,
       1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       1, 1, 1, 2, 2, 2, 2, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2,
       2, 2], dtype=int64)

### Swap Step

As mentioned in the introduction, in this step, for each medoid  $j$  and each data point  $i$  associated to  $j$  swap  $j$  and  $i$  and compute the total cost of the configuration (that is the average dissimilarity of  $i$  to all the data points, $\sum s_{ij}$). Select the medoid  $j$  with the lowest cost of the configuration.

That is, for each cluster, search if any of the points in the cluster decreases the average dissimilarity coefficient. Select the point that decreases this coefficient the most as the medoid for this cluster. 

Dissimilarity is given by __Minkowski(p)__, so you could use functions created in earlier exercises.

Given a clustering (i.e. a set of points and their medoids) and order of $p$, computing the new medoid of each cluster. 

__Note__: The computation of medoids is __NOT__ the same as that of cluster means. 

__Reference__: This link could be helpful in better understanding the swap step!
https://www.youtube.com/watch?v=OWpRBCrx5-M

In [219]:
def update_medoids(X, medoids, p):
    
    S = compute_d_p(points, medoids, p)
    labels = assign_cluster_labels(S)
        
    out_medoids = medoids
                
    for i in set(labels):
        
        avg_dissimilarity = np.sum(compute_d_p(points, medoids[i], p))

        cluster_points = points[labels == i]
        
        for datap in cluster_points:
            new_medoid = datap
            new_dissimilarity= np.sum(compute_d_p(points, datap, p))
            
            if new_dissimilarity < avg_dissimilarity :
                avg_dissimilarity = new_dissimilarity
                
                out_medoids[i] = datap
                
    return out_medoids

Lastly, a function to check whether the medoids have "moved," given two instances of the medoid values. It accounts for the fact that the order of medoids may have changed.

In [220]:
def has_converged(old_medoids, medoids):
    return set([tuple(x) for x in old_medoids]) == set([tuple(x) for x in medoids])

Put all of the preceding building blocks together to implement the PAM k-medoids algorithm.

1. __Initialize__: randomly select $k$ of the $m$ data points as the medoids
2. __Assignment step__: Associate each data point to the closest medoid.
3. __Update step__: For each medoid $j$ and each data point $i$ associated to $j$ swap $j$ and $i$ and compute the total cost of the configuration (that is, the average dissimilarity of $i$ to all the data points associated to $j$). Select the medoid $j$ with the lowest cost of the configuration.
Repeat alternating steps 2 and 3 until there is __no change__ in the assignments.

In [221]:
def kmedoids(X, k, p, starting_medoids=None, max_steps=np.inf):
    if starting_medoids is None:
        medoids = init_medoids(X, k)
    else:
        medoids = starting_medoids
        
    converged = False
    labels = np.zeros(len(X))
    i = 1
    while (not converged) and (i <= max_steps):
        old_medoids = medoids.copy()
        
        S = compute_d_p(X, medoids, p)
        
        labels = assign_cluster_labels(S)
        
        medoids = update_medoids(X, medoids, p)
        
        converged = has_converged(old_medoids, medoids)
        print(medoids)
        i += 1
    return labels

In [222]:
clustering = kmedoids(points, 3, 2)
df['clustering'] = clustering

[[-0.1325144   0.17702523 -0.05753628]
 [ 0.07542113  0.19612962 -0.0780123 ]
 [ 0.4060866  -0.19028041  0.009633  ]]
[[-0.1325144   0.17702523 -0.05753628]
 [ 0.07542113  0.19612962 -0.0780123 ]
 [ 0.4060866  -0.19028041  0.009633  ]]


Final cluster label assignment by K-Medoids method is:

In [223]:
clustering

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 2, 1, 0, 1, 0,
       0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 2, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 2, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2], dtype=int64)

In [224]:
# Test cell: `kmedoids_test`

# Helper Functions

def mark_matches(a, b, exact=False):
    """
    Given two Numpy arrays of {0, 1} labels, returns a new boolean
    array indicating at which locations the input arrays have the
    same label (i.e., the corresponding entry is True).
    
    This function can consider "inexact" matches. That is, if `exact`
    is False, then the function will assume the {0, 1} labels may be
    regarded as the same up to a swapping of the labels. This feature
    allows
    
      a == [0, 0, 1, 1, 0, 1, 1]
      b == [1, 1, 0, 0, 1, 0, 0]
      
    to be regarded as equal. (That is, use `exact=False` when you
    only care about "relative" labeling.)
    """
    assert a.shape == b.shape
    a_int = a.astype(dtype=int)
    b_int = b.astype(dtype=int)
    all_axes = tuple(range(len(a.shape)))
    assert ((a_int == 0) | (a_int == 1) | (a_int == 2)).all()
    assert ((b_int == 0) | (b_int == 1) | (b_int == 2)).all()
    
    exact_matches = (a_int == b_int)
    if exact:
        return exact_matches

    assert exact == False
    num_exact_matches = np.sum(exact_matches)
    if (2*num_exact_matches) >= np.prod (a.shape):
        return exact_matches
    return exact_matches == False # Invert

def count_matches(a, b, exact=False):
    """
    Given two sets of {0, 1} labels, returns the number of mismatches.
    
    This function can consider "inexact" matches. That is, if `exact`
    is False, then the function will assume the {0, 1} labels may be
    regarded as similar up to a swapping of the labels. This feature
    allows
    
      a == [0, 0, 1, 1, 0, 1, 1]
      b == [1, 1, 0, 0, 1, 0, 0]
      
    to be regarded as equal. (That is, use `exact=False` when you
    only care about "relative" labeling.)
    """
    matches = mark_matches(a, b, exact=exact)
    return np.sum(matches)

n_matches = count_matches(wine_type, df['clustering'])
print(n_matches,
      "matches out of",
      len(df), "possible",
      "(~ {:.1f}%)".format(100.0 * n_matches / len(df)))

(145, 'matches out of', 178, 'possible', '(~ 81.5%)')


Depending on the initial seed, the result would fluctuate but at this seed, 81.5% accuracy is achieved. Let's visualize the model's assignment:

In [225]:
trace2 = go.Scatter3d(
    x=PCAdf['principal component 1'],
    y=PCAdf['principal component 2'],
    z=PCAdf['principal component 3'],
    mode='markers',
    marker=dict(
        size=6,
        color=clustering,                # set color to an array/list of desired values
        colorscale='Viridis',   # choose a colorscale
        opacity=0.8
    )
)

data1 = [trace2]

fig1 = go.Figure(data=data, layout=layout)
iplot(fig1)

THE END!